In [34]:
import numpy as np
import pandas as pd

from plotly.graph_objs import *
import re

#plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.grid_objs import Grid, Column #sliders
import plotly.graph_objs as go #horizontal bar charts
import plotly.plotly as py
init_notebook_mode(connected=True)

#LDA
import pyLDAvis
import pyLDAvis.gensim
import gensim

pyLDAvis.enable_notebook()

/usr/local/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning:

`scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.



In [22]:
tweets = pd.read_csv('../../finalprojdata/tweets.csv')
users = pd.read_csv('../../finalprojdata/users.csv')
fulltweets = pd.read_csv('../../finalprojdata/fulltweets2.csv')

In [3]:
users['Date'] = pd.to_datetime(users['created_at'])
users = users[pd.notnull(users['created_at'])]
users['Date'] = users['Date'].apply(lambda x: x.strftime('%Y-%m'))

## Who are the users?

In [6]:
usersname = pd.DataFrame(users.name.str.split(' ',1).tolist(),
                                   columns = ['first','last'])

usersnamesum = usersname.groupby('first',as_index=False).size().reset_index(name='counts')

usersnamesum = usersnamesum.sort_values('counts', ascending=False).head(20)

In [15]:
usersnamesum = usersname.groupby('last',as_index=False).size().reset_index(name='counts')

usersnamesum = usersnamesum.sort_values('counts', ascending=False).head(20)

data = [go.Bar(
            x=usersnamesum['counts'],
            y=usersnamesum['last'],
            orientation = 'h'
)]

layout = Layout(
    title = "Russian Bot First Names",
    yaxis=dict(title='Name'),
    xaxis = dict(title='Number of Accounts')
)

fig = Figure(data=data, layout=layout)
iplot(fig)

In [18]:
usersnamesum = usersname.groupby('first',as_index=False).size().reset_index(name='counts')

usersnamesum = usersnamesum.sort_values('counts', ascending=False).head(20)

data = [go.Bar(
            x=usersnamesum['counts'],
            y=usersnamesum['first'],
            orientation = 'h'
)]

layout = Layout(
    title = "Russian Bot First Names",
    yaxis=dict(title='Name'),
    xaxis = dict(title='Number of Accounts')
)

fig = Figure(data=data, layout=layout)
iplot(fig)

We can see that the Russian Bot names tend to fall under a few categories: typical American names, foreign names, and a few generic names (such as New ... or ). With names such as these, it's often quite difficult to determine which are bots or real individuals. 

## When were the accounts made?

In [19]:
userssum = users.groupby('Date',as_index=False).size().reset_index(name='counts')
users.sort_values('Date')

,id,location,name,followers_count,statuses_count,time_zone,verified,lang,screen_name,description,created_at,favourites_count,friends_count,listed_count,Date
0,1.871082e+07,near Utah Ave & Lighthouse an,Robby Delaware,304.0,11484.0,Pacific Time (US & Canada),False,en,RobbyDelaware,"I support the free movement of people, ideas a...",Wed Jan 07 04:38:02 +0000 2009,17.0,670.0,13.0,2009-01
317,3.887576e+07,St-Petersburg,Olga,14.0,219.0,St. Petersburg,False,ru,Leecory,here i am~,Sat May 09 15:58:34 +0000 2009,0.0,6.0,0.0,2009-05
326,8.758894e+07,NaN,Одинокий Джордж,1267.0,61735.0,Moscow,False,ru,anzgri,Никто.,Thu Nov 05 01:44:48 +0000 2009,75.0,36.0,111.0,2009-11
1,1.003451e+08,still ⬆️Block⤵️Corner⬇️street,#Ezekiel2517✨...,1053.0,31858.0,NaN,False,en,SCOTTGOHARD,CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...,Tue Dec 29 23:15:22 +0000 2009,2774.0,1055.0,35.0,2009-12
2,2.471657e+08,"Chicago, IL",B E C K S T E R✨,650.0,6742.0,Mountain Time (US & Canada),False,en,Beckster319,Rebecca Lynn Hirschfeld Actress.Model.Writer.A...,Fri Feb 04 06:38:45 +0000 2011,7273.0,896.0,30.0,2011-02
3,2.495389e+08,NaN,Chris Osborne,44.0,843.0,NaN,False,en,skatewake1994,NaN,Wed Feb 09 07:38:44 +0000 2011,227.0,154.0,1.0,2011-02
4,4.496897e+08,NaN,Рамзан Кадыров,94773.0,10877.0,Moscow,False,ru,KadirovRussia,"Пародийный аккаунт. Озвучиваю то, что политика...",Thu Dec 29 11:31:09 +0000 2011,0.0,7.0,691.0,2011-12
5,4.718689e+08,"Санкт-Петербург, Россия",Маргарита Саваж,23305.0,18401.0,Volgograd,False,ru,MargoSavazh,Честь имею. Нагло врать не умею.,Mon Jan 23 10:26:25 +0000 2012,1480.0,8021.0,231.0,2012-01
374,5.356905e+08,NaN,LinaLuna,785.0,9394.0,Volgograd,False,ru,Lmarinays,НЛО,Sat Mar 24 20:41:43 +0000 2012,627.0,202.0,3.0,2012-03
6,1.039581e+09,Amerika,Dark Nally,22.0,22603.0,Jakarta,False,id,darknally,uptodate news!!!!,Thu Dec 27 13:06:58 +0000 2012,0.0,40.0,16.0,2012-12


In [20]:
trace0 = Bar(
    name = "Accounts Created Over Time",
    x=userssum.Date,
    y=userssum.counts)
data = Data([trace0])

layout = Layout(
    title = "Number of Russian Bots Created Between 2009 and 2017",
    yaxis=dict(
        title='Number of Incidents',
        range=[0, 100]
    ),
   
    xaxis = dict(
        title='Year',
        range = ['2009-01','2017-1']
    )
)
fig = Figure(data=data, layout=layout)
iplot(fig)

Of the 454 accounts deemed to be fake Russian accounts, we can see the creation of the bots started in 2009 and reached its peak of creations in 2013 before slowly lessening all the way to the start of 2017. 

# Where are they from?

In [21]:
users.location.value_counts()

USA                              93
United States                    23
Москва                           16
Москва, Россия                    9
Chicago, IL                       7
Atlanta                           6
US                                6
Atlanta, GA                       5
Санкт-Петербург                   3
New York, NY                      3
Houston, TX                       3
Baltimore, MD                     3
Texas, USA                        3
Los Angeles, CA                   3
St Louis, MO                      2
Washington, DC                    2
Baton Rouge, LA                   2
Омск                              2
NY                                2
Philadelphia, PA                  2
Miami, FL                         2
New York, USA                     2
Мурманск                          2
still ⬆️Block⤵️Corner⬇️street     1
Estados Unidos                    1
Калуга, Россия                    1
СПб                               1
Georgia, USA                

Note: we did take a look at where the fake bots "originated" from and discovered that of the 454 values, approximately half the values were missing. Of the 287 locations listed, 124 were listed as some form of "United States", 61 were listed as a large metropolitan cities in the United States (ie: San Francisco, New York, Atlanta, Los Angeles), and the remaining 100 values were in foreign

## how influential are these users

In [38]:
trace = go.Scatter(
    y = users.followers_count,
    x = users.statuses_count,
    mode = 'markers',
    text = users.name,
    size = 8
)

data = [trace]

layout = Layout(
    title = "Russian Bot Followers and Tweet Count",
    xaxis=dict(title='Number of Followers'
    ),
   
    yaxis = dict(
        title='Number of Tweets'
    ),
    
    annotations=[dict(x=20576,y=61609,xref='x',
            yref='y',text='Jenna Abrams'),
                dict(x=42438,y=60897,xref='x',
            yref='y',text='New York City Today')]
)
fig = Figure(data=data, layout=layout)

iplot(fig)

In general, we can see the number of Tweets increasing with the number of followers. 

# What are they saying?

In [23]:
# heat map showing the days and hours of users

We can see that the map shows that the bots are most active on

In [35]:
# LDA topics 
dictionary = gensim.corpora.Dictionary.load('dictionary.dict')
corpus = gensim.corpora.MmCorpus('corpus.mm')
ldamodel = gensim.models.LdaModel.load('topic.model')

In [36]:
data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
data

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
17     5.652078        1       1  0.269390 -0.038418
21     4.082652        1       2  0.207074 -0.011219
28     4.003842        1       3 -0.044191  0.224570
27     3.965532        1       4  0.087466  0.131443
9      3.850252        1       5  0.190262  0.006419
25     3.680442        1       6  0.166511 -0.017891
11     3.526953        1       7 -0.027825  0.066462
14     3.453641        1       8  0.083642 -0.104792
24     3.438111        1       9 -0.027985  0.111117
4      3.321473        1      10 -0.078400  0.164542
0      3.231427        1      11 -0.017994 -0.076229
13     3.230222        1      12  0.058620 -0.002203
23     3.223801        1      13 -0.023846  0.128801
12     3.188437        1      14 -0.003502  0.105461
7      3.184742        1      15 -0.043826 -0.092018
19     3.098035        1      16 -0.062619 -0.077927
8      3.086067        1      17 -0.027137 -0.051963
29     3.080491        1      18 -0.049124 -0.019002
3      3.076198        1      19  0.050652  0.003056
16     3.033868        1      20 -0.097865 -0.055391
10     3.015934        1      21 -0.080050 -0.095464
22     3.012110        1      22 -0.038976 -0.059701
20     3.003440        1      23 -0.085357 -0.049674
5      2.999063        1      24 -0.072599 -0.006369
15     2.984936        1      25 -0.050851 -0.035763
1      2.951488        1      26 -0.080618 -0.057125
18     2.946492        1      27 -0.084379  0.010517
2      2.922456        1      28 -0.059199 -0.082429
6      2.913232        1      29 -0.059057  0.003384
26     2.842588        1      30  0.001783 -0.022193, topic_info=       Category          Freq             Term         Total  loglift  logprob
term                                                                          
235874  Default  23817.000000            trump  23817.000000  30.0000  30.0000
31953   Default   9612.000000          clinton   9612.000000  29.0000  29.0000
172447  Default   9021.000000          hillary   9021.000000  28.0000  28.0000
38966   Default   4989.000000             dont   4989.000000  27.0000  27.0000
48151   Default   3588.000000             tcot   3588.000000  26.0000  26.0000
56046   Default   4473.000000           donald   4473.000000  25.0000  25.0000
27477   Default   3921.000000         politics   3921.000000  24.0000  24.0000
183588  Default   3521.000000          america   3521.000000  23.0000  23.0000
10757   Default   4863.000000             like   4863.000000  22.0000  22.0000
36914   Default   3093.000000        president   3093.000000  21.0000  21.0000
238384  Default   2893.000000             know   2893.000000  20.0000  20.0000
135144  Default   2797.000000             vote   2797.000000  19.0000  19.0000
22149   Default   2611.000000            never   2611.000000  18.0000  18.0000
233129  Default   2505.000000             good   2505.000000  17.0000  17.0000
119324  Default   2670.000000             want   2670.000000  16.0000  16.0000
223723  Default   2392.000000            white   2392.000000  15.0000  15.0000
136538  Default   2599.000000            pjnet   2599.000000  14.0000  14.0000
74981   Default   2432.000000             love   2432.000000  13.0000  13.0000
66034   Default   2584.000000             maga   2584.000000  12.0000  12.0000
176039  Default   2247.000000         midnight   2247.000000  11.0000  11.0000
231205  Default   2439.000000            think   2439.000000  10.0000  10.0000
7915    Default   2240.000000             cant   2240.000000   9.0000   9.0000
136102  Default   7366.000000            obama   7366.000000   8.0000   8.0000
30154   Default   2033.000000             back   2033.000000   7.0000   7.0000
35015   Default   1938.000000         american   1938.000000   6.0000   6.0000
190213  Default   4277.000000             news   4277.000000   5.0000   5.0000
216484  Default   2735.000000             make   

The bots tend to talk in topics about

# How are they gaining attention?

In [26]:
# sentiment over time, polarity, (favorites / retweets, sentiments, polarity)

In [ ]:
# What techniques and jargon do these bots use? (ngram, word cloud)

#sentiment and followers
#created account expo
#tweets over time
#sentiment over time

In [16]:
usersfulltweet = fulltweets.groupby(by='user_key', as_index=False).mean().sort_values("user_key", ascending=True)
usersfulltweet.head()

,user_key,Unnamed: 0,user_id,retweet_count,favorite_count,id,followers_count,statuses_count,verified,favourites_count,...,created_strMonth,created_strWeek,created_strDay,created_strDayofweek,created_strDayofyear,created_strHour,created_strMinute,created_strSecond,created_strElapsed,Sentiment
0,4mysquad,110440.581395,4.036537e+09,6.200000,3.400000,4.036537e+09,4733.0,1489.0,False,544.0,...,9.581395,40.162791,16.162791,3.000000,276.813953,12.581395,27.255814,28.558140,1.452748e+09,0.011156
1,6druz,101714.800000,2.534876e+09,0.000000,0.000000,2.534876e+09,70.0,2024.0,False,2751.0,...,10.800000,44.200000,10.400000,2.800000,309.200000,8.800000,23.400000,18.800000,1.478268e+09,0.181667
2,_nickluna_,101153.089902,1.649967e+09,0.009202,0.027607,1.649967e+09,1457.0,2100.0,False,2001.0,...,6.376385,25.793606,15.271921,2.818297,178.928775,12.969294,29.470085,23.068376,1.471865e+09,0.039198
3,_sashalapin,111467.416667,2.478384e+09,0.000000,0.000000,2.478384e+09,2219.0,1237.0,False,345.0,...,10.666667,44.250000,15.416667,3.333333,310.083333,7.833333,30.166667,28.333333,1.478341e+09,0.112500
4,aantiracist,95512.459459,1.709314e+09,0.000000,0.000000,1.709314e+09,897.0,1498.0,False,1785.0,...,8.351351,33.702703,12.405405,3.189189,236.108108,14.027027,31.891892,33.405405,1.471972e+09,0.049174


## Case Study - top 15 users

In [49]:
subfulltweets = fulltweets.groupby('user_key', as_index=False).sum().sort_values(by='statuses_count')
subusers = subfulltweets.user_key[0:20].values

In [70]:
subfulltweets.head()

,user_key,Unnamed: 0,user_id,retweet_count,favorite_count,id,followers_count,statuses_count,verified,favourites_count,...,created_strMonth,created_strWeek,created_strDay,created_strDayofweek,created_strDayofyear,created_strHour,created_strMinute,created_strSecond,created_strElapsed,Sentiment
273,peeonhillary,47665,7.484650e+17,0.0,0.0,7.484650e+17,30.0,4.0,False,0.0,...,7,28,14,3,196,16,5,13,1468512313,0.00
177,joachimbuchwitz,137154,7.528498e+17,0.0,0.0,7.528498e+17,1.0,5.0,False,13.0,...,7,29,21,3,203,7,19,22,1469085562,0.00
124,georevieweurope,18874,7.565538e+17,0.0,0.0,7.565538e+17,37.0,16.0,False,3.0,...,10,39,2,6,276,10,21,30,1475403690,0.00
199,krustyps,69772,2.579368e+09,0.0,0.0,2.579368e+09,186.0,63.0,False,73.0,...,10,41,10,0,284,20,41,15,1476132075,0.35
137,heyitsglor,679676,1.524666e+10,0.0,0.0,1.524666e+10,444.0,84.0,False,84.0,...,48,186,36,30,1314,79,133,76,8822942056,2.15


In [73]:
subfulltweets2 = fulltweets[fulltweets.user_key.isin(subusers)][['user_key', 'created_strYear', 'followers_count', 'Sentiment', 'statuses_count']]

In [74]:
years_from_col = set(subfulltweets2['created_strYear'])
years_ints = sorted(list(years_from_col))
years = [str(year) for year in years_ints]

In [75]:
continents = []
for continent in subfulltweets2['user_key']:
    if continent not in continents: 
        continents.append(continent)

In [76]:
#make grid
columns = []
for year in years:
    for continent in continents:
        dataset_by_year = subfulltweets2[subfulltweets2['created_strYear'] == int(year)]
        dataset_by_year_and_cont = subfulltweets2[subfulltweets2['user_key'] == continent]
        for col_name in dataset_by_year_and_cont:
            # each column name is unique
            column_name = '{year}_{continent}_{header}_gapminder_grid'.format(
                year=year, continent=continent, header=col_name
            )
            a_column = Column(list(dataset_by_year_and_cont[col_name]), column_name)
            columns.append(a_column)

In [77]:
#url = py.grid_ops.upload(grid, 'gapminder_grid'+str(time.time()), auto_open=False)

figure = {
    'data': [],
    'layout': {},
    'frames': [],
    'config': {'scrollzoom': True}
}

figure['layout']['xaxis'] = {'title': 'Life Expectancy', 'gridcolor': '#FFFFFF'}
figure['layout']['yaxis'] = {'title': 'GDP per Capita', 'type': 'log', 'gridcolor': '#FFFFFF'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['plot_bgcolor'] = 'rgb(223, 232, 243)'


In [78]:
figure['layout']['sliders'] = {'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'text-before-value-on-display',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0 }

In [79]:
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [80]:

year = 2014
for continent in continents:
    data_dict = {
        'xsrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='lifeExp'
        )),
        'ysrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='gdpPercap'
        )),
        'mode': 'markers',
        'textsrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='country'
        )),
        'marker': {
            'sizemode': 'area',
            'sizeref': 200000,
            'sizesrc': grid.get_column_reference(col_name_template.format(
                 year=year, continent=continent, header='pop'
            )),
            'color': custom_colors[continent]
        },
        'name': continent
    }
    figure['data'].append(data_dict)

PlotlyError: Whoops, that column name doesn't match any of the column names in your grid. You must pick from ['2015_angela_lattke_Unnamed: 0_gapminder_grid', '2015_angela_lattke_user_id_gapminder_grid', '2015_angela_lattke_user_key_gapminder_grid', '2015_angela_lattke_retweet_count_gapminder_grid', '2015_angela_lattke_retweeted_gapminder_grid', '2015_angela_lattke_favorite_count_gapminder_grid', '2015_angela_lattke_text_gapminder_grid', '2015_angela_lattke_source_gapminder_grid', '2015_angela_lattke_hashtags_gapminder_grid', '2015_angela_lattke_posted_gapminder_grid', '2015_angela_lattke_mentions_gapminder_grid', '2015_angela_lattke_id_gapminder_grid', '2015_angela_lattke_location_gapminder_grid', '2015_angela_lattke_name_gapminder_grid', '2015_angela_lattke_followers_count_gapminder_grid', '2015_angela_lattke_statuses_count_gapminder_grid', '2015_angela_lattke_time_zone_gapminder_grid', '2015_angela_lattke_verified_gapminder_grid', '2015_angela_lattke_lang_gapminder_grid', '2015_angel